TWO towers approach


In [ ]:
import pandas as pd
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.api.layers import Concatenate, Dense, Dropout, Input
from keras.api.models import Model
from keras.api.optimizers import Adam
from keras.saving import load_model
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_absolute_error, root_mean_squared_error, r2_score
import json
from tensorflow.keras.models import load_model
import sys
sys.path.append('../data')
import load_data
from tensorflow.keras.models import load_model
import numpy as np

2025-05-17 02:09:31.510965: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-17 02:09:31.513871: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-17 02:09:31.520724: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747440571.531525   59268 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747440571.534701   59268 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747440571.544504   59268 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [2]:
small_matrix, big_matrix, item_categories, item_features, social_network, user_features, captions   = load_data.load_data()

item_features = item_features_agg = item_features.groupby("video_id").agg({
    "play_cnt": "sum",
    "share_cnt": "sum",
    "download_cnt": "sum",
    "comment_cnt": "sum",
    "upload_type": "first",
    "author_id": "first",
    "video_duration": "first"
})
item_features["video_duration"] = item_features["video_duration"].fillna(item_features["video_duration"].median())
item_features = item_features.dropna()
item_features = item_features.drop_duplicates()
all_categories = set(cat for sublist in item_categories["feat"] for cat in sublist)
for cat in all_categories:
    item_features[f'cat_{cat}'] = item_categories["feat"].apply(lambda x: int(cat in x))


Loading data...
Data loaded.
Cleaning data...
Data cleaned.


In [6]:
# Implement two tower model



# User tower
user_input = Input(shape=(user_features.shape[1],), name='user_input')
user_dense = Dense(128, activation='relu')(user_input)
user_dense = Dropout(0.2)(user_dense)
user_dense = Dense(64, activation='relu')(user_dense)
user_dense = Dense(32, activation='relu')(user_dense)
user_dense = Dropout(0.2)(user_dense)

# Item tower
item_input = Input(shape=(item_features.shape[1],), name='item_input')
item_dense = Dense(128, activation='relu')(item_input)
item_dense = Dropout(0.2)(item_dense)
item_dense = Dense(64, activation='relu')(item_dense)
item_dense = Dense(32, activation='relu')(item_dense)
item_dense = Dropout(0.2)(item_dense)

# Concatenate towers
merged = Concatenate()([user_dense, item_dense])
merged = Dense(64, activation='relu')(merged)
merged = Dropout(0.2)(merged)
output = Dense(1, activation='linear', name='output')(merged)

model = Model(inputs=[user_input, item_input], outputs=output)
model.compile(optimizer=Adam(), loss='mse', metrics=['mae'])

lr_scheduler = ReduceLROnPlateau( monitor="val_loss", factor=0.5, patience=3, min_lr=1e-6, verbose=1)
early_stopping = EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)



In [4]:
# Encode the labels of string columns
item_features["upload_type"] = LabelEncoder().fit_transform(item_features["upload_type"])
user_features["user_active_degree"] = LabelEncoder().fit_transform(user_features["user_active_degree"])
# Standardize features by removing the mean and scaling to unit variance
user_features = StandardScaler().fit_transform(user_features)
item_features = StandardScaler().fit_transform(item_features)
watch_ratios = StandardScaler().fit_transform(big_matrix[["watch_ratio"]].values)

# Split into train and test

user_features_train = user_features[big_matrix["user_id"]]
item_features_train = item_features[big_matrix["video_id"]]
y_train = watch_ratios


user_features_test = user_features[small_matrix["user_id"]]
item_features_test = item_features[small_matrix["video_id"]]
y_test = StandardScaler().fit_transform(small_matrix[["watch_ratio"]].values)

In [ ]:
model.fit(
    x=[user_features_train, item_features_train],
    y=y_train,
    validation_data=([user_features_test, item_features_test], y_test),
    epochs=10,
    batch_size=128,
    callbacks=[lr_scheduler, early_stopping]
)

model.save()



Epoch 1/10
90352/90352 ━━━━━━━━━━━━━━━━━━━━ 200s 2ms/step - loss: 0.9436 - mae: 0.3324 - val_loss: 0.9387 - val_mae: 0.2956 - learning_rate: 0.0010
Epoch 2/10
90352/90352 ━━━━━━━━━━━━━━━━━━━━ 195s 2ms/step - loss: 0.9376 - mae: 0.3260 - val_loss: 0.9400 - val_mae: 0.3055 - learning_rate: 0.0010
Epoch 3/10
90352/90352 ━━━━━━━━━━━━━━━━━━━━ 195s 2ms/step - loss: 0.9683 - mae: 0.3258 - val_loss: 0.9381 - val_mae: 0.3021 - learning_rate: 0.0010
Epoch 4/10
90352/90352 ━━━━━━━━━━━━━━━━━━━━ 196s 2ms/step - loss: 0.9288 - mae: 0.3253 - val_loss: 0.9374 - val_mae: 0.3028 - learning_rate: 0.0010
Epoch 5/10
90352/90352 ━━━━━━━━━━━━━━━━━━━━ 197s 2ms/step - loss: 0.9463 - mae: 0.3256 - val_loss: 0.9387 - val_mae: 0.2989 - learning_rate: 0.0010
Epoch 6/10
90352/90352 ━━━━━━━━━━━━━━━━━━━━ 198s 2ms/step - loss: 0.9543 - mae: 0.3253 - val_loss: 0.9372 - val_mae: 0.3053 - learning_rate: 0.0010
Epoch 7/10
90352/90352 ━━━━━━━━━━━━━━━━━━━━ 197s 2ms/step - loss: 0.9433 - mae: 0.3252 - val_loss: 0.9414 - val_

In [ ]:

# Load the model
two_tower_model = load_model("two-towers.keras", custom_objects={'tf': tf})

inverse_scaler = StandardScaler().fit(big_matrix[["watch_ratio"]].values)

y_true = inverse_scaler.inverse_transform(y_test).flatten()

predictions = model.predict([user_features_test, item_features_test])
y_pred = inverse_scaler.inverse_transform(predictions)


140456/140456 ━━━━━━━━━━━━━━━━━━━━ 65s 463us/step


In [16]:
# Evaluate the model
mae = mean_absolute_error(y_true, y_pred)
rmse = root_mean_squared_error(y_true, y_pred)
r2 = r2_score(y_true, y_pred)

print(f"Evaluate the model")
print(f"True values: {y_true[:5]}")
print(f"Predicted values: {y_pred[:5]}")
print(f"MAE: {mae}")
print(f"RMSE: {rmse}")
print(f"R2: {r2}")


Evaluate the model
True values: [0.71234272 2.17924934 2.37223467 0.51962865 0.33643166]
Predicted values: [[1.4275737 ]
 [1.4261898 ]
 [0.90504426]
 [1.2559516 ]
 [0.9100992 ]]
MAE: 0.5125399806729959
RMSE: 1.6252481923620308
R2: 0.06278182109012642


In [ ]:
# Conclusion for the Two Towers Model
print("Conclusion for the Two Towers Model:")
print(f"The Two Towers model was trained to predict user-video interactions using user and item features.")
print(f"Evaluation Metrics:")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared (R2): {r2}")

if r2 > 0.5:
    print("The model demonstrates good predictive performance.")
elif r2 > 0.2:
    print("The model shows moderate predictive performance.")
else:
    print("The model's predictive performance is limited, and further optimization may be needed.")

## Conclusion for the Two Towers Model

The Two Towers model was designed to predict user-video interactions by leveraging both user and item features. The model was trained and evaluated using a dataset of user interactions with videos.

### Evaluation Metrics:
- **Mean Absolute Error (MAE):** 0.5125
    - This metric indicates the average absolute difference between the predicted and actual values. A lower value suggests better performance.
- **Root Mean Squared Error (RMSE):** 1.6252
    - RMSE measures the square root of the average squared differences between predictions and actual values. It penalizes larger errors more heavily than MAE.
- **R-squared (R2):** 0.0628
    - R2 represents the proportion of variance in the dependent variable that is predictable from the independent variables. A low R2 score indicates that the model explains only a small portion of the variance in the data.

### Insights:
- The low R2 score suggests that the model's predictive performance is limited and that it struggles to capture the underlying patterns in the data.
- The MAE and RMSE values provide a measure of the prediction errors, with RMSE emphasizing larger errors.



### Final Thoughts:
While the Two Towers model demonstrates some ability to predict user-video interactions, its current performance suggests significant room for improvement. Future iterations should focus on addressing the identified limitations through model refinement, feature engineering, and alternative approaches. With these enhancements, the model's predictive accuracy and generalization capabilities can be improved.
